In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
baseURL = 'https://www.bbc.com'

In [3]:
request = requests.get('https://www.bbc.com/urdu')

In [4]:
request.status_code

200

In [5]:
data = None

In [6]:
if request.status_code == 200:
    data = request.text

In [7]:
soup = BeautifulSoup(data, 'html5lib')

In [8]:
soup.prettify()

'<!DOCTYPE html>\n<html class="no-js" dir="rtl" lang="ur">\n <head>\n  <meta content="IE=edge" data-react-helmet="true" http-equiv="X-UA-Compatible"/>\n  <meta charset="utf-8" data-react-helmet="true"/>\n  <meta content="noodp,noydir" data-react-helmet="true" name="robots"/>\n  <meta content="#B80000" data-react-helmet="true" name="theme-color"/>\n  <meta content="width=device-width, initial-scale=1, minimum-scale=1" data-react-helmet="true" name="viewport"/>\n  <meta content="BBC News اردو" data-react-helmet="true" name="apple-mobile-web-app-title"/>\n  <meta content="BBC News اردو" data-react-helmet="true" name="application-name"/>\n  <meta content="تازہ ترین خبروں، ویڈیوز اور آڈیوز کے لیے بی بی سی اردو پر آئیے۔ بی بی سی اردو دنیا بھر کی خبروں کے حصول کے لیے ایک قابلِ اعتماد ویب سائٹ ہے۔" data-react-helmet="true" name="description"/>\n  <meta content="100004154058350" data-react-helmet="true" property="fb:admins"/>\n  <meta content="1609039196070050" data-react-helmet="true" property

In [9]:
sections_soup = soup.find_all('a', {'class' : 'StyledLink-sc-158x7z5-2'})
sections = []
for section in sections_soup:
    if section.text ==  'ویڈیو' or section.text == 'پنجابی ويڈیو':
        continue
    sections.append(section)


In [10]:
sections

[<a class="StyledLink-sc-158x7z5-2 dxndRL" href="/urdu"><span class="StyledSpan-sc-158x7z5-4 ktTFbu" role="text"><span class="VisuallyHiddenText-sc-1yjwwa5-0 ldFJPH">موجودہ صفحہ<!-- -->, </span>صفحۂ اول</span></a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/pakistan">پاکستان</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/regional">آس پاس</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/world">ورلڈ</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/sport">کھیل</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/entertainment">فن فنکار</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/science">سائنس</a>]

In [11]:
for section in sections:
    url = baseURL+section.get('href')
    sec_stories = BeautifulSoup(requests.get(url).text, 'html.parser')
    print(url)

https://www.bbc.com/urdu
https://www.bbc.com/urdu/pakistan
https://www.bbc.com/urdu/regional
https://www.bbc.com/urdu/world
https://www.bbc.com/urdu/sport
https://www.bbc.com/urdu/entertainment
https://www.bbc.com/urdu/science


In [12]:
dataset = {}
story_no = 0

In [13]:
for section in sections:
    url = baseURL+section.get('href')
    count = 0
    while(True):
        sec_stories = BeautifulSoup(requests.get(url).text, 'html.parser')
        if(section.text == 'آس پاس'):
            stories = sec_stories.find_all('div', {'class':'eagle-item faux-block-link'})
        else:
            stories = sec_stories.find_all('li', {'class':['StoryPromoLiBase-sc-171lqjd-0','lx-stream__post-container']})
        heading = "Heading: "
        summary = "Summery: "
        for story in stories: 
            if(story != None):
                count += 1
                if section.text == 'موجودہ صفحہ, صفحۂ اول':
                    heading = story.find('a',{'class':'StyledLink-sc-16i2p1z-2'}).text
                elif section.text == 'آس پاس':
                    heading = story.find('h3',{'class':'title-link__title'}).span.text
                else:
                    heading = story.find('a',{'class':'qa-heading-link'}).span.text
                if section.text == 'موجودہ صفحہ, صفحۂ اول':
                    summary = story.find('p',{'class':'Summary-sc-1dvfmi3-4'}).text
                elif section.text == 'آس پاس':
                    summary = story.find('p',{'class':'eagle-item__summary'}).text
                else:
                    summary = story.find('p', {'class':['lx-media-asset-summary', 'qa-story-summary']}).text
            story_no += 1
            dataset[story_no] = [section.text, heading, summary]
            print("Section: ", section.text)
            print("Heading:",heading)
            print("Summary: ",summary)
            print(count, " Stories")
            if count >=18 and section.text ==  'موجودہ صفحہ, صفحۂ اول':
                break
        next = sec_stories.find('a',{'rel':'next'})
        if next and next.get('href') and count<30:
            url = baseURL+next.get('href')
        else:
            break

Section:  موجودہ صفحہ, صفحۂ اول
Heading: لائیو امریکی صدارتی انتخاب 2020: ابتدائی متوقع نتائج میں کسی ریاست میں کامیاب پارٹی تبدیل نہیں ہوئی
Summary:  امریکہ کے 59ویں صدارتی انتخاب میں دس کروڑ ووٹ انتخاب کے دن سے پہلے ہی ڈالے جا چکے ہیں اور ملک کی تقریباً تمام ریاستوں میں ووٹنگ کا عمل مکمل ہو چکا ہے۔ 31 ریاستوں کے متوقع نتائج جاری کر دیے گئے ہیں جس کے مطابق جو بائیڈن کو صدر ٹرمپ پر سبقت حاصل ہے ۔
1  Stories
Section:  موجودہ صفحہ, صفحۂ اول
Heading: امریکی صدارتی انتخاب کی رات کن باتوں پر نظر رکھنے کی ضرورت ہے؟
Summary:  امریکہ میں آج اگلے چار برسوں کے لیے ملکی سیاست کا تعین تو ہوگا ہی لیکن اس کے اثرات پوری دنیا پر پڑیں گے، تو ایسی صورت میں آپ کو کِن باتوں کو مدِنظر رکھنے کی ضرورت ہے؟
2  Stories
Section:  موجودہ صفحہ, صفحۂ اول
Heading: تین نومبر، ملک میں ایمرجنسی اور کھینچا تانی
Summary:  ایسا نہیں تھا کہ تین نومبر سنہ 2007 کا دن آیا اور اس وقت کے فوجی صدر پرویز مشرف کو اچانک خیال آیا کہ کوئی ماورائے آئین کام کیا جائے اور اُنھوں نے اپنے اس خواب کی تعبیر کرتے ہوئے ملک میں ایمرجنسی کے نفاذ

Section:  پاکستان
Heading: بریگیڈیئر اعجاز شاہ اور آسیب کا سایہ
Summary:  کراچی میں امریکی صحافی ڈینیئل پرل کے اغوا کے بعد جب پورے ملک کی پولیس اور خفیہ ادارے عمر شیخ کو تلاش کر رہے تھے تو اعجاز شاہ نے اسے اپنے ہی گھر سے برآمد کر لیا تھا: محمد حنیف کا کالم
11  Stories
Section:  پاکستان
Heading: جب پاکستان کے پہلے صدر کو ’ملک کے مفاد میں‘ جلاوطن کیا گیا
Summary:  گھڑی کی سوئیوں نے اِدھر دس بجنے کا اعلان کیا اُدھر رات کے اندھیرے میں ایوب خان کا سُرخ وسپید چہرہ چمکا، انھوں نے صدر مملکت اسکندر مرزا کو سیلیوٹ کیا اور کہا ’سب ٹھیک ہے سر، آپریشن مکمل ہو چکا ہے۔‘
12  Stories
Section:  پاکستان
Heading: پاکستان سمیت مختلف ممالک کورونا وائرس کی ’دوسری لہر‘ کا مقابلہ کیسے کر رہے ہیں؟
Summary:  جانز ہاپکنز یونیورسٹی کے اعداد و شمار کے مطابق اب تک دنیا بھر میں 46,312,467 افراد میں کورونا کی تشخیص ہوئی ہے جبکہ اموات کی تعداد 1,198,000 ہے۔ بعض یورپی ممالک میں لاک ڈاؤن جبکہ کچھ ممالک میں حفاظتی ہدایات پر عملدرآمد پر دوبارہ زور دیا جا رہا ہے۔
13  Stories
Section:  پاکستان
Heading: یہ کیسے یقینی بنایا جا

Section:  ورلڈ
Heading: جب تہران میں امریکی سفارتخانے پر ایرانی مظاہرین نے قبضہ کر لیا
Summary:  آج سے ٹھیک 41 برس قبل، یعنی چار نومبر 1979، اُس واقعے کی ابتدا ہوئی جو آگے چل کر دنیا کی سب سے طاقتور ریاستوں میں سے ایک امریکہ اور دنیا کی قدیم ترین سلطنتوں میں سے ایک ایران کے درمیان سفارتی تعلقات کے خاتمے کا باعث بنا۔
1  Stories
Section:  ورلڈ
Heading: امریکی صدارتی انتخاب کی رات کن باتوں پر نظر رکھنے کی ضرورت ہے؟
Summary:  امریکہ میں آج اگلے چار برسوں کے لیے ملکی سیاست کا تعین تو ہوگا ہی لیکن اس کے اثرات پوری دنیا پر پڑیں گے، تو ایسی صورت میں آپ کو کِن باتوں کو مدِنظر رکھنے کی ضرورت ہے؟
2  Stories
Section:  ورلڈ
Heading: ویانا: ’حملہ آور کو گذشتہ دسمبر ہی جیل سے رہا کیا گیا تھا‘
Summary:  آسٹریا کے وزیر داخلہ کا کہنا ہے کہ ہلاک ہونے والا حملہ آور شدت پسند تنظیم داعش کا ہمدرد تھا۔ پولیس کے مطابق حملہ آور نے جسم پر نقلی دھماکہ خیز مواد والی بیلٹ پہن رکھی تھی۔ دوسرے حملہ آور کی تلاش جاری ہے۔
3  Stories
Section:  ورلڈ
Heading: ریپ کا شکار خاتون جنھیں ’اپنے چہرے پر ماسک اس کا ہاتھ محسوس ہوتا

Section:  کھیل
Heading: خوش بختی سے خجالت تک، چار ہی گیندوں میں
Summary:  پاکستان نے اس میچ کو سپر اوور تک زندہ رکھا، قابلِ تعریف بات ہے۔ لیکن سپر اوور میں جو کچھ ہوا، قابلِ غور بھی ہے اور باعثِ ندامت بھی: سمیع چوہدری کا کالم
1  Stories
Section:  کھیل
Heading: زمبابوے کی قسمت کب کھلے گی؟
Summary:  اگر زمبابوے کی ٹیم 28 سال تک انٹرنیشنل کرکٹ کونسل کی رکن رہ کر بھی ایک قدم آگے نہیں بڑھ سکی تو اس سے یقیناً کونسل کی اپنی قابلیت بھی مشکوک ٹھہرتی ہے: پڑھیے پاکستان اور زمبابوے کی سیریز پر سمیع چوہدری کا کالم۔
2  Stories
Section:  کھیل
Heading: پاکستانی امپائر علیم ڈار کا ایک اور عالمی ریکارڈ
Summary:  پاکستانی کرکٹ امپائر علیم ڈار آج پاکستان اور زمبابوے کے درمیان راولپنڈی میں دوسرے ون ڈے انٹرنیشنل میں امپائرنگ کے لیے میدان میں اترے تو یہ ان کا 210 واں ون ڈے انٹرنیشنل میچ تھا جو ایک ریکارڈ ہے۔
3  Stories
Section:  کھیل
Heading: ٹیلر کی کہانی کا ’اینٹی کلائمیکس‘
Summary:  برینڈن ٹیلر اپنے تجربے کی روشنی میں ٹیم کی کشتی گہرے پانیوں سے نکال کر دھیرے دھیرے ساحل کے قریب لے آئے مگر اچانک دوسرے کنارے

Section:  فن فنکار
Heading: جیمز بانڈ کا کردار نبھانے والے شان کونری 90 برس کی عمر میں وفات پا گئے
Summary:  جیمز بانڈ کا کردار نبھانے والے سر شان کونری 90 سال کی عمر میں وفات پا گئے ہیں۔ ان کی وفات کی تصدیق ان کے گھر والوں نے کی ہے۔
1  Stories
Section:  فن فنکار
Heading: سری لنکن کرکٹر مرلی دھرن کی زندگی پر بننے والی فلم تنازعات کا شکار کیوں؟
Summary:  سری لنکا کے سابق آف سپنر متھیا مرلی دھرن کا کہنا ہے کہ ان کی زندگی پر مبنی فلم تنازعات اور تنقید کے باوجود ضرور بنائی جائے گی۔ بی بی سی کی نالینی سواتھاسن نے اس حوالے سے مرلی دھرن سے گفتگو کی۔
2  Stories
Section:  فن فنکار
Heading: پاکستانی اداکار علی خان اب برطانوی فلم میں نظر آئیں گے
Summary:  پاکستانی ڈراموں اور فلموں سے شہرت حاصل کرنے والے اداکار علی خان نئی برطانوی فلم 'مغل موگلی' میں اہم کردار نبھا رہے ہیں۔ یہ فلم 30 اکتوبر کو برطانیہ اور آئرلینڈ کے سنیما گھروں میں نمائش کے لیے پیش کی جائے گی۔
3  Stories
Section:  فن فنکار
Heading: کم کارڈیشیئن کی نجی جزیرے پر ’عاجزانہ‘ سالگرہ پارٹی پر کڑی تنقید
Summary:  ریئلٹی ٹی وی سٹار کا کہنا

Section:  سائنس
Heading: ’کیا عورت کو بس ماں، بیوی ہی بننا ہوتا ہے، اُن کی اپنی کوئی زندگی نہیں‘
Summary:  کئی صارفین یہ بھی پوچھتے نظر آتے ہیں کہ کیا لڑکیاں دنیا میں صرف بیوی بننے اور بچے پیدا کرنے کے لیے آئی ہیں؟ ان کی زندگی کا اور کوئی مقصد نہیں؟
1  Stories
Section:  سائنس
Heading: کیا امریکہ انڈیا کو پاکستان کے حساس علاقوں کے نقشوں تک رسائی فراہم کر سکتا ہے؟
Summary:  انڈیا اور امریکہ کے درمیان ہونے والے تازہ دفاعی معاہدے پر چین اور پاکستان دونوں کو تحفظات ہیں اور ماہرین کا کہنا ہے کہ تازہ معاہدہ خطے میں مزید تناؤ کا باعث بنے گا۔
2  Stories
Section:  سائنس
Heading: ’خودکش‘ خلائی مشن پر جانے والی ’لائیکا‘ خلا میں کتنے دن زندہ رہی؟
Summary:  تین نومبر 1957 کو ایک بڑا قدم اٹھاتے ہوئے روس نے سپوتنک ٹو خلائی گاڑی کے ساتھ ایک کتیا کو پہلی دفعہ زمین کے گرد مدار میں بھیجا جس کا نام ’لائیکا‘ تھا۔ آئیے اس بہادر اور حیران کن جانور کے بارے میں جاننے کی کوشش کرتے ہیں جو انسانی خلائی سفر کے باب کا اہم کردار ہے۔
3  Stories
Section:  سائنس
Heading: غالب اور سر سید: مشکل پسندی سے سہل پسندی کا سفر
S

In [14]:
dataset

{1: ['موجودہ صفحہ, صفحۂ اول',
  'لائیو امریکی صدارتی انتخاب 2020: ابتدائی متوقع نتائج میں کسی ریاست میں کامیاب پارٹی تبدیل نہیں ہوئی',
  'امریکہ کے 59ویں صدارتی انتخاب میں دس کروڑ ووٹ انتخاب کے دن سے پہلے ہی ڈالے جا چکے ہیں اور ملک کی تقریباً تمام ریاستوں میں ووٹنگ کا عمل مکمل ہو چکا ہے۔ 31 ریاستوں کے متوقع نتائج جاری کر دیے گئے ہیں جس کے مطابق جو بائیڈن کو صدر ٹرمپ پر سبقت حاصل ہے ۔'],
 2: ['موجودہ صفحہ, صفحۂ اول',
  'امریکی صدارتی انتخاب کی رات کن باتوں پر نظر رکھنے کی ضرورت ہے؟',
  'امریکہ میں آج اگلے چار برسوں کے لیے ملکی سیاست کا تعین تو ہوگا ہی لیکن اس کے اثرات پوری دنیا پر پڑیں گے، تو ایسی صورت میں آپ کو کِن باتوں کو مدِنظر رکھنے کی ضرورت ہے؟'],
 3: ['موجودہ صفحہ, صفحۂ اول',
  'تین نومبر، ملک میں ایمرجنسی اور کھینچا تانی',
  'ایسا نہیں تھا کہ تین نومبر سنہ 2007 کا دن آیا اور اس وقت کے فوجی صدر پرویز مشرف کو اچانک خیال آیا کہ کوئی ماورائے آئین کام کیا جائے اور اُنھوں نے اپنے اس خواب کی تعبیر کرتے ہوئے ملک میں ایمرجنسی کے نفاذ کا اعلان کر دیا ہو۔'],
 4: ['موجودہ صفحہ, صفحۂ اول',
 

In [15]:
df = pd.DataFrame.from_dict(dataset, orient='index', columns=['Label', 'Heading', 'Summary'])

In [16]:
df

,Label,Heading,Summary
1,"موجودہ صفحہ, صفحۂ اول",لائیو امریکی صدارتی انتخاب 2020: ابتدائی متوقع...,امریکہ کے 59ویں صدارتی انتخاب میں دس کروڑ ووٹ ...
2,"موجودہ صفحہ, صفحۂ اول",امریکی صدارتی انتخاب کی رات کن باتوں پر نظر رک...,امریکہ میں آج اگلے چار برسوں کے لیے ملکی سیاست...
3,"موجودہ صفحہ, صفحۂ اول",تین نومبر، ملک میں ایمرجنسی اور کھینچا تانی,ایسا نہیں تھا کہ تین نومبر سنہ 2007 کا دن آیا ...
4,"موجودہ صفحہ, صفحۂ اول",خوش بختی سے خجالت تک، چار ہی گیندوں میں,پاکستان نے اس میچ کو سپر اوور تک زندہ رکھا، قا...
5,"موجودہ صفحہ, صفحۂ اول",امریکی صدارتی انتخاب 2020: کیا انتخاب کی رات ف...,کیا امریکی صدارتی انتخاب کی رات کو ہی انتخاب ک...
6,"موجودہ صفحہ, صفحۂ اول",امریکی صدارتی انتخاب کے نتائج: آج تین ممکنہ شہ...,امریکی صدارتی انتحاب 2020 کی مہم ایک غیر معمول...
7,"موجودہ صفحہ, صفحۂ اول",ریپ کا شکار خاتون جنھیں ’اپنے چہرے پر ماسک اس ...,ریپ کا شکار ہونے والی برطانوی خاتون جارجینا فی...
8,"موجودہ صفحہ, صفحۂ اول",کراچی: مندر پر حملہ، نامعلوم افراد کے خلاف توہ...,پاکستان کے سب سے بڑے شہر کراچی میں مقامی ہندو ...
9,"موجودہ صفحہ, صفحۂ اول",ویانا: ’حملہ آور کو گذشتہ دسمبر ہی جیل سے رہا ...,آسٹریا کے وزیر داخلہ کا کہنا ہے کہ ہلاک ہونے و...
10,"موجودہ صفحہ, صفحۂ اول",امریکہ میں عوام اپنا صدر کیسے چنتے ہیں؟,کیا آپ جانتے ہیں کہ امریکہ میں صدارتی انتخاب م...


In [18]:
df.to_csv('Dataset.csv', encoding='utf-32', columns=['Label', 'Heading', 'Summary'], sep=';')